# Instructions: 

1.) You need to create an account on Genius.com and get an API key in order to run this code.

2.) You need to download the artists dataset from here: https://www.kaggle.com/datasets/pieca111/music-artists-popularity?resource=download

In [1]:
import lyricsgenius as lg
import re

import numpy as np
import pandas as pd

import unicodedata
import pickle

In [123]:
genius = lg.Genius('WV-fVabMd4cvDVcYlAaMT55eWzT_c7dZ8AP8dVkM6anvs-LZpR7Mq388L5OSzcgx', timeout = 60, 
                   skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"],
                   remove_section_headers=True, sleep_time=0.3, retries=20)

In [3]:
df = pd.read_csv('artists.csv')
print(df.shape)
print(df.columns)

(1466083, 10)
Index(['mbid', 'artist_mb', 'artist_lastfm', 'country_mb', 'country_lastfm',
       'tags_mb', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm',
       'ambiguous_artist'],
      dtype='object')


/Users/gordid/pyrosetta_environment/ENTER/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#Remove any ambiguous artist
df = df[df['ambiguous_artist'] == False]
df.shape

(1423980, 10)

In [5]:
#Filter by USA/UK/Canda/Ireland/Australia (English speaking)
print(df['country_mb'].unique())
print(len(df['country_mb'].unique()))
print(len(df['country_lastfm'].unique()))

df = df[(df['country_mb'] == 'United Kingdom') | (df['country_mb'] == 'United States') | 
        (df['country_mb'] == 'Canada') | (df['country_mb'] == 'Ireland') | (df['country_mb'] == 'Australia')]
df.shape

['United Kingdom' 'United States' 'France' 'Ireland' 'Canada' 'Australia'
 'Colombia' 'Iceland' 'Senegal' 'Norway' 'Jamaica' 'Germany' 'Sweden'
 'Netherlands' 'South Africa' 'Austria' 'Spain' 'Finland' 'New Zealand'
 'Poland' 'Italy' nan 'Belgium' 'Russia' 'Brazil' 'Greece' 'Denmark'
 'Hungary' 'British Virgin Islands' 'Czech Republic' 'Cuba' 'South Korea'
 'Romania' 'Japan' 'Mexico' 'Barbados' 'Monaco' 'China' 'Israel' 'Jersey'
 'Argentina' 'Estonia' 'Switzerland' 'Bermuda' 'India' 'Moldova'
 'Portugal' 'Cape Verde' 'Mali' 'Malaysia' 'Nigeria' 'Faroe Islands'
 'Ivory Coast' 'Guatemala' 'Soviet Union' 'Bahamas' 'Dominican Republic'
 'Saint Vincent and The Grenadines' 'Ukraine' 'Slovenia' 'Ethiopia'
 'Belarus' 'Turkey' 'Uruguay' 'Armenia' 'Venezuela' 'Algeria' 'Pakistan'
 'Croatia' 'Slovakia' 'Serbia' 'Philippines' 'Liechtenstein' 'Chile'
 'Lithuania' 'Egypt' 'Cameroon' 'Luxembourg' 'Taiwan' 'Morocco' 'Peru'
 'Panama' 'Iran' 'Tunisia' 'Jordan' 'Kosovo' 'Nicaragua' 'Georgia'
 'U.S. Virgi

(266210, 10)

In [6]:
#Filter out any artists without a genre label from both sources
print(sum(df['tags_mb'].isnull()))
print(sum(df['tags_lastfm'].isnull()))

df = df[(df['tags_mb'].notnull()) & (df['tags_lastfm'].notnull())].reset_index()
df.shape

222743
188478


(26410, 11)

In [7]:
#Find all Rap artists
rap_index = []

tags1 = ['rap','rapper', 'hardcore hip hop', 'alternative hip hop', 'pop rap', 'gangsta rap']
tags2 = ['underground hip hop', 'alternative rap', 'trap rap']
tags3 = ['folk', 'techno', 'pop rock', 'indie rock', 'hard rock', 
             'dance-pop', 'alternative rock', 'soul', 'electro', 'crunk']


for i in range(df.shape[0]):
      
    tags_mb = df['tags_mb'][i].split("; ")
    tags_fm = df['tags_lastfm'][i].split("; ")
    
    bool_mb = any(x in tags_mb for x in tags1)
    
    bool_fm = any(x in tags_fm for x in tags2)
    
    bool_negate = any(x in tags_mb for x in tags3) + (('pop rap' not in tags_mb) & ('electropop' in tags_mb))
    
    bool_final = (bool_mb + bool_fm)*(not bool_negate)  
    
    if (bool_final):
        rap_index.append(i)
        
df_rap = df.iloc[rap_index].reset_index()
df_rap = df_rap.drop(np.where(df_rap['artist_mb'] == 'Trey Songz')[0][0])
df_rap = df_rap.reset_index(drop=True)
df_rap.shape

(834, 12)

In [24]:
#rap3 is 100 songs of artists 61-100
artists_rap = df_rap['artist_mb']

#songsList_rap3 = []

for i in range(84,101):
    
    print(i)
    
    songs = genius.search_artist(artists_rap[i], max_songs=100, sort='popularity', per_page=20, 
               get_full_info=True, allow_name_change=True, include_features=False)
    
    songsList_rap3.append(songs)

84
Searching for songs by MF DOOM...

Song 1: "Doomsday"
Song 2: "Beef Rapp"
Song 3: "Rapp Snitch Knishes"
Song 4: "One Beer"
Song 5: "That’s That"
Song 6: "Hoe Cakes"
Song 7: "Rhymes Like Dimes"
Song 8: "Deep Fried Frenz"
Song 9: "Potholderz"
Song 10: "Gazzillion Ear"
Song 11: "Vomitspit"
Song 12: "? (Questions)"
Song 13: "Kon Karne"
Song 14: "Kookies"
Song 15: "Kon Queso"
Song 16: "All Outta Ale"
Song 17: "Gas Drawls"
Song 18: "Batty Boyz"
Song 19: "Guinnesses"
Song 20: "Red and Gold"
Song 21: "Tick, Tick..."
Song 22: "Cellz"
Song 23: "My Favorite Ladies"
Song 24: "The Finest"
Song 25: "Hey!"
Song 26: "Dead Bent"
Song 27: "Absolutely"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Madvillain-sickfit-lyrics
Song 28: "Go with the Flow"
Song 29: "Ballskin"
Song 30: "Who You Think I Am?"
Song 31: "Angelz"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Madvillain-do-not-fi

In [33]:
print(len(songsList_rap3))
with open("songsList_rap3", "wb") as fp:   #Pickling
    pickle.dump(songsList_rap3, fp)

40


In [107]:
df_country = df.drop(rap_index)
df_country = df_country.reset_index(drop=True)

#Find all Rap artists
country_index = []

tags1 = ['country']
#tags2 = ['underground hip hop', 'alternative rap', 'trap rap']
tags3 = ['contemporary r&b', 'reggae', 'indie folk', 'hard rock', 'rnb', 'new wave', 
         'electropop']

for i in range(df_country.shape[0]):
      
    tags_mb = df_country['tags_mb'][i].split("; ")
    #tags_fm = df['tags_lastfm'][i].split("; ")
    
    bool_mb = any(x in tags_mb for x in tags1)
    
    #bool_fm = any(x in tags_fm for x in tags2)
    
    bool_negate = any(x in tags_mb for x in tags3)
    
    bool_final = (bool_mb)*(not bool_negate)  
    
    if (bool_final):
        country_index.append(i)

df_country = df_country.iloc[country_index].reset_index()
df_country = df_country.drop(np.where(df_country['artist_mb'] == 'Bob Dylan')[0][0]).reset_index(drop=True)
df_country = df_country.drop(np.where(df_country['artist_mb'] == 'Elvis Presley')[0][0]).reset_index(drop=True)
df_country = df_country.drop(np.where(df_country['artist_mb'] == 'Sheryl Crow')[0][0])
df_country = df_country.reset_index(drop=True)
print(df_country.shape)
df_country[0:50]

(440, 12)


,level_0,index,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,52,57,d43d12a1-2dc9-4257-a2fd-0a3bb1081b86,Johnny Cash,Johnny Cash,United States,United States,rock; folk; american; blues; singer-songwriter...,country; folk; singer-songwriter; classic rock...,2795136.0,104272344.0,False
1,411,524,c80f38a6-9980-485d-997c-5c1a9cbd0d64,Ryan Adams,Ryan Adams,United States,United States,rock; american; country; alternative country,singer-songwriter; Alt-country; rock; indie; f...,1194703.0,41956467.0,False
2,521,658,70f0e309-5418-49b6-a130-666e2f76eecd,Carrie Underwood,Carrie Underwood,United States,United States,pop; country; country rock; pop rock; country ...,country; female vocalists; pop; American Idol;...,1019092.0,20021495.0,False
3,559,704,0eedfc95-e79a-4fd9-832a-cd816a0b3fda,Lady Antebellum,Lady Antebellum,United States,United States,pop; country,country; modern country; country pop; lady ant...,979874.0,14896242.0,False
4,583,736,faabb55d-3c9e-4c23-8779-732ac2ee2c0d,Shania Twain,Shania Twain,Canada,Canada,rock; pop; canadian; country; country rock; da...,country; pop; female vocalists; Canadian; Shan...,949638.0,12452412.0,False
5,593,754,1d543e07-d0d2-4834-a8db-d65c50c2a856,Dolly Parton,Dolly Parton,United States,United States,pop; folk; american; bluegrass; country; count...,country; female vocalists; singer-songwriter; ...,934924.0,10441870.0,False
6,665,857,6e0ae159-8449-4262-bba5-18ec87fa529f,Rascal Flatts,Rascal Flatts,United States,United States,country; country pop; classic pop and rock; co...,country; rascal flatts; country pop; rock; pop...,863015.0,12791713.0,False
7,675,869,abae8575-ec8a-4736-abc3-1ad5093a68aa,Jewel,Jewel,United States,United States,pop; singer-songwriter; country; dance-pop; po...,female vocalists; pop; singer-songwriter; folk...,856428.0,10845855.0,False
8,687,885,6cfd7ffc-824f-4219-8e27-4b9417700f44,Brad Paisley,Brad Paisley,United States,United States,country; country rock; country pop; contempora...,country; modern country; brad paisley; singer-...,846511.0,12044872.0,False
9,701,899,5dca4d22-d6c8-4e70-8ca1-4543e43353c7,Kenny Chesney,Kenny Chesney,United States,United States,country; country pop; contemporary country; si...,country; Kenny Chesney; modern country; rock; ...,840917.0,11641283.0,False


In [125]:
artists_country = ['John Anderson', 'Bonnie Raitt'] + list(df_country['artist_mb'])

#songsList_country = []

for i in range(27,226):
    
    print(i)
    maxSongs = 110
    
    if ((i < 100) & (i > 1)):
        
        maxSongs = 200
    
    songs = genius.search_artist(artists_country[i], max_songs=maxSongs, sort='popularity', per_page=20, 
               get_full_info=True, allow_name_change=True, include_features=False)
    
    songsList_country.append(songs)

27
Searching for songs by Jerry Lee Lewis...

Song 1: "Great Balls of Fire"
Song 2: "Whole Lotta Shakin’ Goin On"
Song 3: "Billy Boy"
Song 4: "High School Confidential"
Song 5: "Chantilly Lace"
Song 6: "Drinking Wine Spo-Dee O’Dee"
Song 7: "Another Place, Another Time"
Song 8: "The Lily of the Valley"
Song 9: "Crazy Arms"
Song 10: "Down the Road a Piece"
Song 11: "Rockin’ My Life Away"
Song 12: "Big Legged Woman"
Song 13: "Wild One"
Song 14: "Too Much to Gain to Lose"
Song 15: "It’ll Be Me"
Song 16: "Middle Age Crazy"
Song 17: "Breathless"
Song 18: "Pink Cadillac"
Song 19: "Old Black Joe"
Song 20: "Thirty-Nine and Holding"
Song 21: "Hadacol Boogie"
Song 22: "Mean Old Man"
Song 23: "You Win Again"
Song 24: "No Headstone on My Grave"
Song 25: "Before the Night Is Over"
Song 26: "What’d I Say"
"Great Balls of Fire (Live)" is not valid. Skipping.
Song 27: "Mean Woman Blues"
Song 28: "I’ll Make It All Up to You"
Song 29: "That Lucky Old Sun"
Song 30: "She Even Woke Me Up to Say Goodbye"
Son

In [129]:
#songsList_country.pop(59)
with open("songsList_country", "wb") as fp:   #Pickling
    pickle.dump(songsList_country, fp)

In [150]:
import math
frac_songs = 0.75
lyricsList_country = []
i = 0
for artist in songsList_country:
        
    count = 0


    if len(artist.songs) < 51:

        numMax = len(artist.songs)
        
    else:
        
        numMax = math.ceil(50 + (len(artist.songs) - 50)*frac_songs)
        
        
    print(i, ":", len(artist.songs), ":", numMax)
          
    for song in artist.songs:
        
        title = cleanLyrics(song.title)
        artist = cleanLyrics(song.artist)
        lyrics = cleanLyrics(song.lyrics)
        
        if len(lyrics) > 250:
        
            lyricsList_country.append([artist, title, lyrics])
            count = count + 1
            
        if count == numMax:
            
            break
            
    i += 1

df_country_final = pd.DataFrame(lyricsList_country, columns = ['Artist', 'Song', 'Lyrics'])
print(df_country_final.shape)

#with open("lyricsList_country", "wb") as fp:   #Pickling
#    pickle.dump(lyricsList_country, fp)

0 : 110
1 : 110
2 : 200
3 : 200
4 : 146
5 : 148
6 : 197
7 : 200
8 : 184
9 : 200
10 : 200
11 : 200
12 : 200
13 : 200
14 : 192
15 : 200
16 : 154
17 : 114
18 : 200
19 : 200
20 : 150
21 : 151
22 : 200
23 : 113
24 : 158
25 : 200
26 : 200
27 : 200
28 : 83
29 : 200
30 : 200
31 : 97
32 : 117
33 : 170
34 : 200
35 : 162
36 : 84
37 : 133
38 : 113
39 : 200
40 : 47
41 : 92
42 : 105
43 : 85
44 : 95
45 : 128
46 : 130
47 : 200
48 : 200
49 : 193
50 : 70
51 : 139
52 : 134
53 : 133
54 : 200
55 : 200
56 : 200
57 : 109
58 : 84
59 : 99
60 : 138
61 : 82
62 : 114
63 : 58
64 : 200
65 : 200
66 : 89
67 : 138
68 : 149
69 : 200
70 : 200
71 : 55
72 : 200
73 : 200
74 : 87
75 : 181
76 : 200
77 : 200
78 : 200
79 : 200
80 : 200
81 : 87
82 : 193
83 : 200
84 : 186
85 : 200
86 : 53
87 : 155
88 : 135
89 : 200
90 : 192
91 : 113
92 : 86
93 : 200
94 : 200
95 : 98
96 : 69
97 : 200
98 : 101
99 : 36
100 : 110
101 : 110
102 : 96
103 : 110
104 : 110
105 : 74
106 : 110
107 : 110
108 : 101
109 : 76
110 : 110
111 : 110
112 : 67
113 :

In [128]:
#remove Jerry Lee Lewis #27
#remove rockabilly
#remove Calexico
#len(songsList_country)

225

In [154]:
df_country_final = df_country_final[df_country_final.Artist != 'Calexico']
df_country_final = df_country_final[df_country_final.Artist != 'Jerry Lee Lewis']
df_country_final = df_country_final[df_country_final.Artist != 'Reverend Horton Heat']
df_country_final = df_country_final[df_country_final.Artist != 'Wanda Jackson']

print(df_country_final.shape)

(21982, 3)


In [155]:
df_country_final = df_country_final.drop_duplicates()
df_country_final.to_csv('df_country_final.csv')
print(df_country_final.shape)

(21887, 3)


In [157]:
print(df_country_final.shape)
df_country_final.to_csv('df_country_final.csv', index=False)

(21887, 3)


In [ ]:
#with open("lyricsList_country", "wb") as fp:   #Pickling
#    pickle.dump(lyricsList_country, fp)

In [133]:
for artist in df_country['artist_mb']:
    
    if "Charlie" in artist:
        
        print(artist)
        print(np.where(df['artist_mb'] == artist))
        print(np.where(df_country['artist_mb'] == artist))
        print(np.where(df_country2['artist_mb'] == artist))

The Charlie Daniels Band
(array([2151]),)
(array([69]),)
(array([72]),)
Charlie Rich
(array([3861]),)
(array([134]),)
(array([137]),)
Charlie Daniels
(array([4863]),)
(array([175]),)
(array([178]),)
Charlie Robison
(array([6980]),)
(array([226]),)
(array([229]),)


In [100]:
num = np.where(df['artist_mb'] == 'Lynyrd Skynyrd')[0][0]

print(df['tags_mb'][num])
print(num)
print(df['tags_lastfm'][num])
print(df['artist_mb'][num])

rock; american; hard rock; blues rock; southern rock; acoustic rock; boogie rock; classic pop and rock; lynyrd skynyrd
184
classic rock; Southern Rock; rock; hard rock; 70s; blues rock; country; blues; lynyrd skynyrd; american; country rock; seen live; rock n roll; oldies; USA; guitar; southern; 60s; classic; blues-rock; 80s; Boogie Rock; alternative; metal; Progressive rock; favorites; male vocalists; florida; free bird
Lynyrd Skynyrd


In [127]:
for i in range(df_country.shape[0]):
    
    tags = df_country['tags_mb'][i].split("; ")
    
    if 'rockabilly' in tags:
        
        print(df_country['artist_mb'][i])
        print(i)   

Johnny Cash
0
Miranda Lambert
22
Jerry Lee Lewis
25
Bill Haley and His Comets
58
Carl Perkins
67
Wanda Jackson
85
Reverend Horton Heat
118
Charlie Rich
134
Sleepy LaBeef
304
Commander Cody & His Lost Planet Airmen
308
Ray Campi
315
Two Tons of Steel
343
Tibby Edwards
392
The Ranch Girls
418
Phil Trigwell
426


In [38]:
print(np.where(df_rap['artist_mb'] == 'Hopsin'))

(array([223]),)


In [367]:
df['tags_mb'][6].split("; ")

'synthpop; pop; american; hip-hop; hip hop; electropop; producer; pop rap; rapper; us; hip hop rnb and dance hall; experimental hip hop; grammy winner; hardcore hip hop'

In [95]:
for i in range(df_country2.shape[0]):
    
    if 'pop rock' in df_country2['tags_mb'][i]:
        
        print(df_country2['artist_mb'][i])
    

Bob Dylan
Elvis Presley
Sheryl Crow
Carrie Underwood
Shania Twain
Jewel
Dixie Chicks
Sugarland
Lonestar
Del Shannon
Uncle Kracker
Jimmy Buffett
The Strumbellas
Wonderland


In [375]:
num = np.where(df['artist_mb'] == 'Iggy Azalea')[0][0]

print(df['tags_mb'][num])
print(num)
print(df['tags_lastfm'][num])
print(df['artist_mb'][num])

electropop; pop rap; hip house; hyphy; trap rap; a filk artist
934
Hip-Hop; rap; pop; hip hop; australia; female vocalists; electronic; australian; dance; rnb; trap; seen live; igloo australia; all things annoying in the world put together into one stupid bitch; Officially Shit; 3; Trash; the worst thing ever to happen to music; Iggy Azalea; chillout; electropop; World Music; untalented; swag; Skitz0rz; uitluisteren artist; holloween skitz0rz; chill; world; female rapper; my dance room; singer-songwriter; dubstep; awful; sunflower; pure trash; disgusting human being; shit; EDM; your ears will bleed; crimes against humanity; horrible songs; female; alternative; alternative hip-hop; All; Rapper; fav888; indie; Queen; underground rap; r&b; female vocal; pop rap; 10s; whore untalented; white trash; charli xcx; heroine pop; queen of rap; white nigga
Iggy Azalea


In [110]:
df_rap = df.iloc[rap_index].reset_index()

for i in range(50):
    
    print(df_rap['artist_mb'][i], ':', df_rap['tags_mb'][i])
    print(df_rap['artist_mb'][i], ':', df_rap['tags_lastfm'][i])
    
    print('/n')

Eminem : turkish; rap; american; hip-hop; hip hop; hiphop; midwest hip-hop; detroit hip-hop; pop rap; horrorcore; rapper; us; pop and chart; eminem; 8 mile; boom bap; v; comedy rap; hardcore hip hop
Eminem : rap; Hip-Hop; Eminem; hip hop; pop; american; detroit; seen live; hiphop; Gangsta Rap; male vocalists; slim shady; rock; alternative; hardcore rap; 00s; 90s; USA; singer-songwriter; Shady; d12; emo; Horrorcore; Aftermath; dance; Michigan; white rap
/n
Kanye West : synthpop; pop; american; hip-hop; hip hop; electropop; producer; pop rap; rapper; us; hip hop rnb and dance hall; experimental hip hop; grammy winner; hardcore hip hop
Kanye West : Hip-Hop; rap; hip hop; rnb; Kanye West; seen live; Gay Fish; american; pop; chicago; hiphop; soul; electronic; gayfish; producer; alternative; male vocalists; 00s; kanye; indie; rock; urban; experimental; dance; r&b; pop rap; alternative rap; electronica; alternative hip-hop; USA; good music; midwest rap
/n
Drake : hip hop; pop rap; rapper; con

In [296]:
print(np.where(df['artist_mb'] == 'Logic'))

for artist in df['artist_mb']:
    
    if "Cardi" in artist:
        
        print(artist)

(array([1712]),)
Craig Cardiff
Cardiacs
Lindsey Cardinale
Cardiac Kidz
Janet Cardiff
Glenn Cardier


In [69]:
tags_mb = []

for tags in df['tags_mb']:
    
    tags_mb = tags_mb + tags.split("; ")

tags_mb = np.array(tags_mb)
unique, counts = np.unique(tags_mb, return_counts=True)
pd.DataFrame((unique[np.argsort(-counts)], counts[np.argsort(-counts)])).T.head(30)

,0,1
0,jazz,3044
1,punk,2816
2,uk,2811
3,rock,2537
4,usa,2225
5,american,1821
6,british,1659
7,england,1592
8,hip hop,1255
9,classic pop and rock,1106


In [71]:
unique, counts = np.unique(tags_mb, return_counts=True)
pd.DataFrame((unique[np.argsort(-counts)], counts[np.argsort(-counts)])).T.head(50)

,0,1
0,jazz,3044
1,punk,2816
2,uk,2811
3,rock,2537
4,usa,2225
5,american,1821
6,british,1659
7,england,1592
8,hip hop,1255
9,classic pop and rock,1106


In [70]:
tags_fm = []

for tags in df['tags_lastfm']:
    
    tags_fm = tags_fm + tags.split("; ")

tags_fm = np.array(tags_fm)
unique, counts = np.unique(tags_fm, return_counts=True)
pd.DataFrame((unique[np.argsort(-counts)], counts[np.argsort(-counts)])).T.head(30)

,0,1
0,seen live,9418
1,rock,7350
2,american,7087
3,All,5793
4,USA,5115
5,alternative,4866
6,pop,4749
7,jazz,4484
8,indie,4459
9,electronic,4182


In [7]:
print(df.columns)

df = df[['artist_mb', 'artist_lastfm', 'country_mb', 'country_lastfm',
       'tags_mb', 'tags_lastfm', 'listeners_lastfm', 'ambiguous_artist']]

df.shape

Index(['mbid', 'artist_mb', 'artist_lastfm', 'country_mb', 'country_lastfm',
       'tags_mb', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm',
       'ambiguous_artist'],
      dtype='object')


(1466083, 8)

In [396]:
genius = lg.Genius('KMbGSLF5Rtq4oUi4_s0QN1veITTO-rrrWXLq-x-oaGQxb69coWe4QffGE9BSCTMH', 
                   skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"],
                   remove_section_headers=True)

In [3]:
songs = (genius.search_artist('Rihanna', max_songs=2, sort='popularity')).songs

Searching for songs by Rihanna...

Song 1: "Work"
Song 2: "Love on the Brain"

Reached user-specified song limit (2).
Done. Found 2 songs.


In [43]:
cleanLyrics(songs[0].lyrics)

" Work, work, work, work, work, work He said me haffi Work, work, work, work, work, work He see me do mi Dirt, dirt, dirt, dirt, dirt, dirt So me put in Work, work, work, work, work, work When you ah guh Learn, learn, learn, learn, learn? Me nuh cyar if him Hurt, hurt, hurt, hurt, hurting Dry! Me ah desert him Nuh time to have you lurking Him ah go act like he nuh like it You know I dealt with you the nicest Nuh body touch me, you nuh righteous Nuh badda, text me in a crisis I believed all of your dreams, adoration You took my heart and my keys and my patience You took my heart on my sleeve for decoration You mistaken my love I brought for you for foundation All that I wanted from you was to give me Something that I never had Something that you've never seen Something that you've never been Mmmmm But I wake up and act like nothing's wrong Just get ready fi  Work, work, work, work, work, work He said me haffi Work, work, work, work, work, work He see me do mi Dirt, dirt, dirt, dirt, dir

In [130]:
def cleanLyrics(lyrics):
    
    lyrics = re.sub(".+?Lyrics", "", lyrics)
    
    if len(lyrics) > 100:
        
        lyrics = lyrics[:-9]
    
    lyrics = re.sub("\n", " ", lyrics)
    lyrics = unicodedata.normalize("NFKD", lyrics)
    lyrics = lyrics.replace(b'\xe2\x80\x94'.decode('utf-8'), '')
    lyrics = lyrics.replace(b'\xe2\x80\x99'.decode('utf-8'), '')
    lyrics = lyrics.encode("ascii", "ignore")
    lyrics = lyrics.decode()
 
    return lyrics

In [ ]:
def get_lyrics(arr, k):
    c = 0
    for name in arr:
        try:
            songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
            s = [song.lyrics for song in songs]
            file.write("\n \n   <|endoftext|>   \n \n".join(s))
            c += 1
            print(f"Songs grabbed:{len(s)}")
        except:
            print(f"some exception at {name}: {c}")

In [34]:
genius.charts(time_period='all_time', chart_genre='country',
              per_page=50, page=10, type_='songs')


In [47]:
a.keys()

dict_keys(['chart_items', 'next_page'])

In [68]:
a = genius.charts(time_period='all_time', chart_genre='country', page=5)

In [71]:
len(a['chart_items'])

10

In [57]:
a['next_page']

In [73]:
genius.search_all('fugazi', per_page=None, page=None)

{'sections': [{'type': 'top_hit',
   'hits': [{'highlights': [],
     'index': 'artist',
     'type': 'artist',
     'result': {'_type': 'artist',
      'api_path': '/artists/21126',
      'header_image_url': 'https://images.genius.com/86b8a6db9e3731b7e374eee2f3490be9.400x400x1.jpg',
      'id': 21126,
      'image_url': 'https://images.genius.com/86b8a6db9e3731b7e374eee2f3490be9.400x400x1.jpg',
      'index_character': 'f',
      'is_meme_verified': False,
      'is_verified': False,
      'name': 'Fugazi',
      'slug': 'Fugazi',
      'url': 'https://genius.com/artists/Fugazi'}},
    {'highlights': [],
     'index': 'album',
     'type': 'album',
     'result': {'_type': 'album',
      'api_path': '/albums/161374',
      'cover_art_thumbnail_url': 'https://images.genius.com/0fe857e2e6424beddd0b26742f77b800.300x300x1.jpg',
      'cover_art_url': 'https://images.genius.com/0fe857e2e6424beddd0b26742f77b800.600x600x1.jpg',
      'full_title': 'Fugazi by Fugazi',
      'id': 161374,
    

In [74]:
genius.search_artist("fugazi", max_songs=100, sort='title', per_page=20, 
               get_full_info=True, allow_name_change=True, include_features=False)

Searching for songs by fugazi...

Changing artist name to 'Fugazi'
Song 1: "23 Beats Off"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Fugazi-afterthought-lyrics
Song 2: "Afterthought"
Song 3: "And the Same"
Song 4: "And the Same - Demo"
Song 5: "Argument"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Fugazi-arpeggiator-lyrics
Song 6: "Arpeggiator"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Fugazi-arpeggiator-demo-lyrics
Song 7: "Arpeggiator Demo"
Song 8: "Back to Base"
Song 9: "Bad Mouth"
Song 10: "Badmouth - Demo"
Song 11: "Bed for the Scraping"
Song 12: "Birthday Pony"
Song 13: "Blueprint"
Song 14: "Break"
Song 15: "Break-In"
Song 16: "Break-In - Demo"
Song 17: "Break-In (SubPop)"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Fugazi-brendan-1-

Artist(id, songs, ...)